In [3]:
!pip install -qU \
    datasets \
    langchain-pinecone langgraph \
    semantic-router serpapi \
    google-search-results

In [57]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import warnings
warnings.filterwarnings("ignore")

region = "us-west-2"
config = Config(
    region_name=region,
    signature_version = "v4",
    retries={
        "max_attempts":3,
        "mode" : "standard",
    }
)
bedrock_rt = boto3.client("bedrock-runtime", config=config)

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

model_kwargs = {
    "max_tokens" : 4096,
    "temperature" : 0.0,
    "stop_sequences" : ["Human"],
}

llm = ChatBedrock(
    client = bedrock_rt,
    model_id = model_id,
    model_kwargs = model_kwargs,
)

In [4]:
from typing import TypedDict,Annotated
from langchain_core.agents import AgentAction
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    input : str
    chat_history : list[BaseMessage]
    intermediate_steps : Annotated[list[tuple[AgentAction,str]],operator.add]

In [7]:
import re
import requests
from langchain_core.tools import tool

@tool("fetch_arxiv")
def fetch_arxiv(arxiv_id : str):
    """
    Gets the abstract from ArXiv paper given the arxiv ID.Useful for
    finding high-level context about a specific paper.
    """
    res = requests.get(
        f"https://export.arxiv.org/abs/{arxiv_id}"
    )

    abstract_pattern = re.compile(
    r'<blockquote class="abstract mathjax">\s*<span class="descriptor">Abstract:</span>\s*(.*?)\s*</blockquote>',
    re.DOTALL
    )

    re_match = abstract_pattern.search(res.text)
    return re_match.group(1)

In [8]:
arxiv_id = "2401.04088"
print(
    fetch_arxiv.invoke(input={"arxiv_id": arxiv_id})
)

We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model.
Mixtral has the same architecture as Mistral 7B, with the difference that each
layer is composed of 8 feedforward blocks (i.e. experts). For every token, at
each layer, a router network selects two experts to process the current state
and combine their outputs. Even though each token only sees two experts, the
selected experts can be different at each timestep. As a result, each token has
access to 47B parameters, but only uses 13B active parameters during inference.
Mixtral was trained with a context size of 32k tokens and it outperforms or
matches Llama 2 70B and GPT-3.5 across all evaluated benchmarks. In particular,
Mixtral vastly outperforms Llama 2 70B on mathematics, code generation, and
multilingual benchmarks. We also provide a model fine-tuned to follow
instructions, Mixtral 8x7B - Instruct, that surpasses GPT-3.5 Turbo,
Claude-2.1, Gemini Pro, and Llama 2 70B - chat model on human benchmarks. Both


In [12]:
import getpass
from serpapi import GoogleSearch

@tool("web_search")
def web_search(query: str):
    """
    Finds general knowledge information using Google Search. Can also be used
    to augment more 'general' knowledge to a previous specialist query.
    """
    serpapi_params = {
    "engine" : "google",
    "api_key" : getpass.getpass()
    }

    search = GoogleSearch({
        **serpapi_params,
        "q" : "coffee"
    })

    results = search.get_dict()["organic_results"]
    contexts = "\n---\n".join(
        ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
    )
    
    return contexts

In [14]:
def format_rag_contexts(matches: list):
    contexts = []
    for x in matches:
        text = (
            f"Title : {x['metadata']['title']}\n"
            f"Content : {x['metadata']['content']}\n"
            f"ArXiv ID : {x['metadata']['content']}\n"
            f"Related Papers: {x['metadata']['references']}\n"
        )
        contexts.append(text)
    context_str = "\n---\n".join(contexts)
    return context_str

@tool("rag_search_filter")
def rag_search_filter(query: str,arxiv_id:str):
    """
    Finds information from our ArXiv database using natural language query
    and a specific ArXiv ID. Allows us to learn more details about a specific paper.
    """
    xq = encoder([query])
    xc = index.query(vector=xq,top_k = 6,include_metadata = True, filter={"arxiv_id" : arxiv_id})
    context_str = format_rag_contexts(xc["matches"])
    return context_str


@tool("rag_search")
def rag_search(query: str):
    """
    Finds specialist information on AI using a natural language query.
    """
    xq = encoder([query])
    xc = index.query(vector = xq,top_k = 2,include_metadata = True)
    context_str = format_rag_contexts(xc["matches"])
    return context_str


In [75]:
@tool("final_answer")
def final_answer(
    introduction: str,
    research_steps: str,
    main_body: str,
    conclusion: str,
    sources: str
):
    """Returns a natural language response to the user in the form of a research
    report. There are several sections to this report, those are:
    - `introduction`: a short paragraph introducing the user's question and the
    topic we are researching.
    - `research_steps`: a few bullet points explaining the steps that were taken
    to research your report.
    - `main_body`: this is where the bulk of high quality and concise
    information that answers the user's question belongs. It is 3-4 paragraphs
    long in length.
    - `conclusion`: this is a short single paragraph conclusion providing a
    concise but sophisticated view on what was found.
    - `sources`: a bulletpoint list provided detailed sources for all information
    referenced during the research process.
    """
    if type(research_steps) is list:
        research_steps = "\n".join([f"- {r}" for r in research_steps])
    if type(sources) is list:
        sources = "\n".join([f"- {s}" for s in sources])
    return ""

In [66]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


system_prompt = """You are the oracle, the great AI decision maker.
Given the user's query you must decide what to do with it based on the
list of tools provided to you.

If you see that a tool has been used (in the scratchpad) with a particular
query, do NOT use that same tool with the same query again. Also, do NOT use
any tool more than twice (ie, if the tool appears in the scratchpad twice, do
not use it again).

You should aim to collect information from a diverse range of sources before
providing the answer to the user. Once you have collected plenty of information
to answer the user's question (stored in the scratchpad) use the final_answer
tool."""


prompt = ChatPromptTemplate.from_messages([
   ("system", system_prompt),
   MessagesPlaceholder(variable_name="chat_history"),
   ("user", "{input}"),
   ("assistant", "scratchpad: {scratchpad}"),
])

In [76]:
from langchain_core.messages import ToolCall, ToolMessage

tools=[
   rag_search_filter,
   rag_search,
   fetch_arxiv,
   web_search,
   final_answer
]

def create_scratchpad(intermediate_steps: list[AgentAction]):
   research_steps = []
   for i, action in enumerate(intermediate_steps):
       if action.log != "TBD":
           research_steps.append(
               f"Tool: {action.tool}, input: {action.tool_input}\n"
               f"Output: {action.log}"
           )
   return "\n---\n".join(research_steps)


oracle = (
   {
       "input": lambda x: x["input"],
       "chat_history": lambda x: x["chat_history"],
       "scratchpad": lambda x: create_scratchpad(
           intermediate_steps=x["intermediate_steps"]
       ),
   }
   | prompt
   | llm.bind_tools(tools, tool_choice="any")
)

In [77]:
inputs = {
    "input": "tell me something interesting about dogs",
    "chat_history": [],
    "intermediate_steps": [],
}
response = oracle.invoke(inputs)
response

ValueError: Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: messages: final assistant content cannot end with trailing whitespace

In [73]:
out.tool_calls[0]["name"]

NameError: name 'out' is not defined